# MIDI2WAV Synthesis

The output of our Stage 2 ML model is a MIDI file representing a drum beat. We want to do 2 things to this MIDI file:

1) Adjust the soundpack for the MIDI file to change the sounds of the individual drums

2) Synthesize the MIDI to a WAV file so it can be played back as audio*
    * I think we can just go straight to raw audio with the FluidSynth executable for linux. See below
    
TODO TASKS:

1) Collect and test a bunch of SoundFont files to provide different timbres and feels. Hopefully we can match files to genres? Can we find a way to mix and match individual sounds from one file to another file? 

2) Try FluidSynth stand-alone on Linux. Try it on the Pi 


## FluidSynth

FluidSynth is a cross-platform Software Synthesizer ( https://github.com/FluidSynth/fluidsynth ). It has a standalone executable that can run on Linux or Windows, and can synthesize a MIDI file to raw audio. It takes in a SoundFont file which acts as a soundpack, providing sounds for each instrument channel. Swapping the SoundFont file allows the MIDI to get synthesize with different instrument sounds. This accomplishes both of our goals set above. 

Based on the above, I think we can install FluidSynth to our local linux environment on the Pi and call it directly. The command-line API returns raw audio (NOT WAV). As far as I know, this is fine, since we will just be directing the audio to output after that(?). 

The following code demonstrates using pyFluidSynth, which is a set of python bindings for the FluidSynth program. For the following code to work, you need to have pyFluidSynth installed in your python environment (should be set if you are using the BB environment included in our github repo) AND you need to have the FluidSynth dlls in the same folder as this notebook. You can download them here https://github.com/FluidSynth/fluidsynth/releases/tag/v2.1.6. This has everything for the actual executable, but we only need the DLLs because the pyFluidsynth library expects them. So I personally copied `bin/*.dll` to this local folder (BandBuddy/ML/) to get it to work. Everything else you can delete. 

Since we likely will just install FluidSynth directly, this is just a sandbox to play around with without having to mess with docker or running on the Pi.

Code is based on a snippet from the jankiest website of all time https://titanwolf.org/Network/Articles/Article?AID=71ea2959-edcc-47f6-ba80-55411ddba85c#gsc.tab=0

In [2]:
import sys
import os
import fluidsynth
import pretty_midi
import scipy
from scipy import io
from scipy.io import wavfile
import numpy as np

# Open some MIDI file
midi_path = os.path.join('..', 'Data', 'groove_dataset', 'drummer1', 'session1','1_funk_80_beat_4-4.mid')
input_midi = pretty_midi.PrettyMIDI(midi_path)

# SF2 soundpack file defines timbres to use for drum channels
sf2_path = os.path.join('..', 'Data', 'sf2_soundpacks', 'GeneralUserGSv1.471.sf2')

# See doc: https://craffel.github.io/pretty-midi/#pretty_midi.PrettyMIDI.fluidsynth
# Synthesize to audio (returns a np array representing the audo), 44100 samplerate is fairly arbitrary
signal = input_midi.fluidsynth(fs=44100, sf2_path=sf2_path)

# Convert from np array to WAV file
m = np.max(np.abs(signal))
sigf32 = (signal/m).astype(np.float32)
scipy.io.wavfile.write("FS_output.wav", 44100, sigf32)